In [8]:
import time

# 시작시간
start_time = time.time()

# 멀티쓰레드 사용하지 않는 경우 (20만 카운트)
def count(name):
    a = 0
    for i in range(1, 50000001):
        a += i
    return a
        
num_list = ['p1', 'p2', 'p3', 'p4']
for num in num_list:
    count(num)
    
print(f"\r--- {time.time() - start_time} seconds ---")

--- 9.153492450714111 seconds ---


In [9]:
import multiprocessing

multiprocessing.cpu_count()

40

In [23]:
import multiprocessing
import time

# 시작시간
start_time = time.time()

# 멀티쓰레드 사용하는 경우 (20만 카운트)
# Pool 사용해서 함수 실행을 병렬
def count(name):
    a = 0
    for i in range(1, 50000001):
        a += i
        
num_list = ['p1', 'p2' ,'p3', 'p4']

# 멀티 쓰레딩 Pool 사용
pool = multiprocessing.Pool(processes=30) # 현재 시스템에서 사용할 프로세스 개수
pool.map(count, num_list)
pool.close()
pool.join()

print(f"--- {time.time() - start_time} seconds ---")

--- 2.6084721088409424 seconds ---


In [32]:
import concurrent

import urllib.request

URLS = ['http://www.foxnews.com/',
        'http://www.cnn.com/',
        'http://europe.wsj.com/',
        'http://www.bbc.co.uk/',
        'http://some-made-up-domain.com/']

def load_url(url, timeout):
    with urllib.request.urlopen(url, timeout=timeout) as conn:
        return conn.read()
    
with concurrent.futures.ThreadPoolExecutor(max_workers=30) as executor:
    future_to_url = {executor.submit(load_url, url, 60): url for url in URLS}
    for future in concurrent.futures.as_completed(future_to_url):
        url  = future_to_url[future]
        try:
            data = future.result()
        except Exception as exc:
            print('%r generated an exception: %s' % (url, exc))
        else:
            print('%r page is %d bytes' % (url, len(data)))

'http://europe.wsj.com/' generated an exception: HTTP Error 404: Not Found
'http://www.foxnews.com/' page is 312438 bytes
'http://www.cnn.com/' page is 1140608 bytes
'http://some-made-up-domain.com/' page is 64668 bytes
'http://www.bbc.co.uk/' page is 271698 bytes


In [33]:
from multiprocessing import Pool, TimeoutError
import time
import os

def f(x):
    return x*x

if __name__ == '__main__':
    # start 4 worker processes
    with Pool(processes=4) as pool:

        # print "[0, 1, 4,..., 81]"
        print(pool.map(f, range(10)))

        # print same numbers in arbitrary order
        for i in pool.imap_unordered(f, range(10)):
            print(i)

        # evaluate "f(20)" asynchronously
        res = pool.apply_async(f, (20,))      # runs in *only* one process
        print(res.get(timeout=1))             # prints "400"

        # evaluate "os.getpid()" asynchronously
        res = pool.apply_async(os.getpid, ()) # runs in *only* one process
        print(res.get(timeout=1))             # prints the PID of that process

        # launching multiple evaluations asynchronously *may* use more processes
        multiple_results = [pool.apply_async(os.getpid, ()) for i in range(4)]
        print([res.get(timeout=1) for res in multiple_results])

        # make a single worker sleep for 10 secs
        res = pool.apply_async(time.sleep, (10,))
        try:
            print(res.get(timeout=1))
        except TimeoutError:
            print("We lacked patience and got a multiprocessing.TimeoutError")

        print("For the moment, the pool remains available for more work")

    # exiting the 'with'-block has stopped the pool
    print("Now the pool is closed and no longer available")

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
0
1
9
25
16
49
36
81
4
64
400
2485
[2484, 2483, 2486, 2485]
We lacked patience and got a multiprocessing.TimeoutError
For the moment, the pool remains available for more work
Now the pool is closed and no longer available
